<a href="https://colab.research.google.com/github/komo135/tradingrl/blob/master/dist_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

In [0]:
!pip install ta

In [0]:
# from dist_td3 import *
from ape_x import *
import os
import multiprocessing as mp
import tensorflow as tf
import numpy as np
import time
import shutil
from multiprocessing import Pool

df = "audpred60.csv"

_ = shutil.copy("/content/drive/My Drive/"+df,"/content")
# sac_one_step.ckpt
# saver_path = "td3.ckpt"
saver_path = "sac"

restore = False

# if restore == True:
#   _ = shutil.copy("/content/drive/My Drive/" + saver_path + ".data-00000-of-00001","/content")
#   _ = shutil.copy("/content/drive/My Drive/" + saver_path + ".index","/content")
#   _ = shutil.copy("/content/drive/My Drive/checkpoint","/content")
  
if restore == True:
  _ = shutil.copy("/content/drive/My Drive/model/" + saver_path + ".data-00000-of-00001","/content")
  _ = shutil.copy("/content/drive/My Drive/model/"+ saver_path + ".index","/content")
  _ = shutil.copy("/content/drive/My Drive/model/checkpoint","/content")

output_size = 3
window_size = 10
size = 120
num_actor = 3
mem_size = size * num_actor * 500
  
def actor_work(queues, num):
  sess = tf.InteractiveSession()
  actor = Actor(sess, df, window_size, num, size, output_size, saver_path, restore)
  actor.run(queues, 10, 1000,100,24,n=4)

# device='/device:GPU:0'
device = '/cpu:0'
  
def leaner_work(queues):
        sess = tf.InteractiveSession()
        leaner = Leaner(sess, df, window_size, output_size,mem_size, device, saver_path, restore)
        leaner.leaner(queues)

In [0]:
%cd /content

queue = mp.Queue()
ps = [mp.Process(target=leaner_work, args=(queue,))]
for i in range(num_actor):
  ps.append(mp.Process(target=actor_work, args=(queue,i)))

for p in ps:
    p.start()
    time.sleep(1)

for p in ps:
    p.join()

/content


/content/drive/My Drive/memory.py:127: RuntimeWarning: divide by zero encountered in double_scalars
  max_weight = (p_min * n) ** (-self.PER_b)


action probability =  [0.36134453781512604, 0.31092436974789917, 0.32773109243697474]
trade accuracy =  0.4
epoch: 10, total rewards: 4928.000000, mean rewards: 993.882353
action probability =  [0.2773109243697479, 0.3697478991596639, 0.3529411764705882]
trade accuracy =  0.47368421052631576
epoch: 20, total rewards: 12588.000000, mean rewards: 2538.756303
action probability =  [0.2689075630252101, 0.42857142857142855, 0.3025210084033614]
trade accuracy =  0.4634146341463415
epoch: 30, total rewards: 8366.000000, mean rewards: 1687.260504
action probability =  [0.25210084033613445, 0.46218487394957986, 0.2857142857142857]
trade accuracy =  0.5238095238095238
epoch: 40, total rewards: 5301.000000, mean rewards: 1069.109244
action probability =  [0.37815126050420167, 0.33613445378151263, 0.2857142857142857]
trade accuracy =  0.32142857142857145
epoch: 50, total rewards: 93.000000, mean rewards: 18.756303
action probability =  [0.42857142857142855, 0.3025210084033613, 0.26890756302521013]

In [0]:
df = pd.read_csv("audpred60.csv")

In [0]:
import ta

In [0]:
rsi = np.asanyarray(ta.rsi(df["Close"])).reshape((-1, 1))

In [0]:
rsi[-30::]

In [0]:
m_d.mean()